In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
# Bring in csv
output_data_file = "weather_pd.csv"

types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,ushuaia,75,AR,1580262621,81,6.00,-54.80,-68.30,14.90
1,lebu,0,CL,1580262773,66,11.71,-37.62,-73.65,4.75
2,mataura,96,NZ,1580262773,69,17.69,-46.19,168.86,3.88
3,nurota,0,UZ,1580262773,92,-2.00,40.56,65.69,4.10
4,albany,90,US,1580262773,92,1.00,42.60,-73.97,3.10
...,...,...,...,...,...,...,...,...,...
564,ust-nera,81,RU,1580262906,99,-40.13,64.57,143.20,1.46
565,dunedin,100,NZ,1580262906,84,16.08,-45.87,170.50,4.27
566,severobaykalsk,100,RU,1580262907,75,-9.64,55.65,109.32,4.94
567,itacare,40,BR,1580262907,83,28.00,-14.28,-39.00,2.10


In [3]:
gmaps.configure(api_key=g_key)

In [15]:
locations = weather_data_df[["Lat", "Lng"]]
# locations

humidity = weather_data_df["Humidity"].astype(float)

# humidity


fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Narrow down the DataFrame:

new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Country","Temp", "Wind Speed", "Cloudiness", "Lat", "Lng"])

max_temp = (new_types_df["Temp"] <= 30) & (new_types_df["Temp"] > 27)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]


,City,Country,Temp,Wind Speed,Cloudiness,Lat,Lng
53,mahibadhoo,MV,27.07,5.65,0,3.78,72.97
162,san policarpo,PH,27.13,1.14,0,12.18,125.51
267,christchurch,NZ,28.89,3.10,0,-43.53,172.63
438,madang,PG,29.20,2.98,0,-5.22,145.80
480,alugan,PH,27.13,1.14,0,12.22,125.48
488,moron,AR,27.78,3.10,0,-34.65,-58.62


In [6]:
hotel_df = pd.DataFrame(new_types_df[max_temp & wind_speed & cloudiness], columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
53,mahibadhoo,MV,3.78,72.97,
162,san policarpo,PH,12.18,125.51,
267,christchurch,NZ,-43.53,172.63,
438,madang,PG,-5.22,145.80,
480,alugan,PH,12.22,125.48,


In [7]:
target_coordinates = f"{hotel_df['Lat']},{hotel_df['Lng']}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [8]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [9]:
hotel_name

['Kuri Beach View Inn',
 'Bahay ni Singkit',
 'Chateau on the Park - Christchurch, a DoubleTree by Hilton',
 'Madang Resort',
 'Bahay ni Singkit',
 'The Palm House Hotel']

In [10]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')

In [11]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))